## 【練習問題】タイタニックの生存予測 | SIGNATE - Data Science Competition  
https://signate.jp/competitions/102/tutorials/20

In [1]:
import pandas as pd
train = pd.read_csv("train.tsv", sep="\t", index_col=0) # 学習用データ
test = pd.read_csv("test.tsv", sep="\t", index_col=0)   # 評価用データ
sample_submit = pd.read_csv("sample_submit.tsv", sep="\t", index_col=0, header=None) # 応募用サンプルファイル

In [2]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,,
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
7,0,3,male,2.0,3,1,21.0750,S
9,1,2,female,14.0,1,0,30.0708,C
11,1,1,female,58.0,0,0,26.5500,S


In [3]:
train['survived'].value_counts()

0    266
1    179
Name: survived, dtype: int64

In [4]:
train = train[["survived","sibsp", "parch", "fare"]]
test = test[["sibsp", "parch", "fare"]]
y = train["survived"] # 目的変数
X = train.drop(["survived"], axis=1) # 目的変数を除いたデータ

In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)

LogisticRegression()

In [6]:
pred = model.predict_proba(test)[:, 1] 

print(pred[:5])

[0.27086714 0.52854033 0.32716076 0.32918889 0.50920521]


In [8]:
sample_submit[1] = pred
sample_submit.to_csv('submit.tsv', header=None, sep='\t')

## 【練習問題】タイタニックの生存予測 | SIGNATE - Data Science Competition
https://signate.jp/competitions/102/tutorials/23

In [9]:
import pandas as pd
train = pd.read_csv("train.tsv", sep="\t", index_col=0) # 学習用データ
test = pd.read_csv("test.tsv", sep="\t", index_col=0)   # 評価用データ
sample_submit = pd.read_csv("sample_submit.tsv", sep="\t", index_col=0, header=None) # 応募用サンプルファイル

In [10]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,,
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
7,0,3,male,2.0,3,1,21.0750,S
9,1,2,female,14.0,1,0,30.0708,C
11,1,1,female,58.0,0,0,26.5500,S


In [11]:
test.head()

,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
5,3,male,NaN,0,0,8.4583,Q
6,1,male,54.0,0,0,51.8625,S


In [12]:
print(train.shape)
print(test.shape)

(445, 8)
(446, 7)


In [13]:
train['survived'].value_counts()

0    266
1    179
Name: survived, dtype: int64

In [14]:
train.describe()

,survived,pclass,age,sibsp,parch,fare
count,445.000000,445.000000,360.000000,445.000000,445.000000,445.000000
mean,0.402247,2.296629,29.211583,0.546067,0.431461,33.959971
std,0.490903,0.834024,14.154300,1.195247,0.850489,52.079492
min,0.000000,1.000000,0.670000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.000000,0.000000,0.000000,7.925000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,15.000000
75%,1.000000,3.000000,37.250000,1.000000,1.000000,31.387500
max,1.000000,3.000000,80.000000,8.000000,5.000000,512.329200


相関係数の計算
相関強：sex,pclass
相関弱：age,embarks

In [15]:
pd.get_dummies(train).corrwith(train["survived"])

survived      1.000000
pclass       -0.358097
age          -0.081394
sibsp        -0.045087
parch         0.079669
fare          0.258605
sex_female    0.559465
sex_male     -0.559465
embarked_C    0.182568
embarked_Q    0.005062
embarked_S   -0.172769
dtype: float64

室クラスごとの生存率 → 客室が高級なほど生存率高い！

In [16]:
train[['pclass', 'survived']].groupby(['pclass']).mean()

,survived
pclass,
1,0.685185
2,0.443299
3,0.258333


性別生存率 → 女性ほど高い！

In [17]:
train[['sex', 'survived']].groupby(['sex']).mean()

,survived
sex,
female,0.775641
male,0.200692


### データ前処理

In [22]:
# 学習/評価データまとめて処理
data = pd.concat([train, test])

# 欠損値補完
data['age'] = data['age'].fillna(data['age'].mean()) 
data['embarked'] = data['embarked'].fillna(data['embarked'].mode()) 

# ダミー化
data = pd.get_dummies(data)

# 学習/評価データに戻す
train = data.loc[train.index]
test = data.loc[test.index]
test = test.drop(["survived"], axis=1)

In [23]:
train.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
id,,,,,,,,,,,
3,1.0,1,35.0,1,0,53.1000,1,0,0,0,1
4,0.0,3,35.0,0,0,8.0500,0,1,0,0,1
7,0.0,3,2.0,3,1,21.0750,0,1,0,0,1
9,1.0,2,14.0,1,0,30.0708,1,0,1,0,0
11,1.0,1,58.0,0,0,26.5500,1,0,0,0,1


In [24]:
y = train["survived"] # 目的変数
X = train.drop(["survived"], axis=1) # 目的変数を除いたデータ

In [25]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
pred = model.predict_proba(test)[:, 1] 

In [27]:
print(pred[:5])

[0.10559994 0.92063833 0.6370289  0.17446418 0.28392059]


In [28]:
sample_submit[1] = pred
sample_submit.to_csv('submit2.tsv', header=None, sep='\t')